## Interaction

Need to reproject .gpkg to EPSG:4326 to ovelay it onto web map

In [ ]:
%pip install ipyleaflet geopandas rasterio shapely folium ipywidgets

### 1. Display Info

In [ ]:
# --- Paths ---
site_path = "sample_data/"
img_path = site_path + "random_oilpalm.tif"
# crowns_path = site_path + "filtered/filtered_by_knn_crowns.gpkg"
crowns_path = site_path + "crowns_exported.gpkg" # predicted_crowns_map15.gpkg , crowns_exported.gpkg
overlay_img_path = site_path + "overlay_rgb.png"


In [ ]:
# --- Imports ---
import rasterio
import geopandas as gpd
import numpy as np
from shapely.geometry import box, mapping, shape
from shapely.ops import transform
import pyproj
from PIL import Image
import json
from ipyleaflet import (
    Map, GeoJSON, Marker, Popup, 
    basemap_to_tiles, basemaps, ImageOverlay, LayersControl, TileLayer,
    WidgetControl, MarkerCluster
)
from ipywidgets import HTML, Layout
# --- Save RGB overlay as PNG from GeoTIFF ---
def save_rgb_overlay_as_png(img_path, output_img="overlay.png"):
    with rasterio.open(img_path) as src:
        img = src.read()
        bounds = src.bounds
        crs = src.crs
        
        if img.shape[0] >= 3:
            rgb = np.stack([img[0], img[1], img[2]], axis=-1)
        else:
            raise ValueError("Need at least 3 bands for RGB")
        
        # Normalize for better display
        rgb = rgb.astype(np.float32)
        rgb_min, rgb_max = np.percentile(rgb[rgb > 0], (2, 98))
        rgb = np.clip((rgb - rgb_min) / (rgb_max - rgb_min), 0, 1)
        rgb = (rgb * 255).astype(np.uint8)
        
        image = Image.fromarray(rgb)
        image.save(output_img)
        
        return output_img, bounds, crs

overlay_img_path, utm_bounds, utm_crs = save_rgb_overlay_as_png(img_path)

# --- Convert UTM bounds to WGS84 ---
def reproject_bounds_to_wgs84(bounds, src_crs):
    project = pyproj.Transformer.from_crs(src_crs, "EPSG:4326", always_xy=True).transform
    return transform(project, box(*bounds)).bounds

wgs84_bounds = reproject_bounds_to_wgs84(utm_bounds, utm_crs)
lon_min, lat_min, lon_max, lat_max = wgs84_bounds

# --- Load crowns and reproject to WGS84 ---
crowns = gpd.read_file(crowns_path)
print(crowns)
print(crowns.crs)
if crowns.crs != "EPSG:4326":
    crowns = crowns.to_crs("EPSG:4326")

crowns["id"] = crowns.index
crowns_json = crowns.__geo_interface__

for i, feat in enumerate(crowns_json["features"]):
    feat["id"] = i

# --- Initialize Map ---
center_lat = (lat_min + lat_max) / 2
center_lon = (lon_min + lon_max) / 2

m = Map(center=(center_lat, center_lon), zoom=18, max_zoom=20)
m.layout.width = '100%'
m.layout.height = '900px'

# --- Base layers ---
google_sat = TileLayer(
    url="http://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}",
    attribution="Google Satellite",
    name="Google Satellite",
    max_zoom=20
)
m.add_layer(google_sat)

esri_sat = basemap_to_tiles(basemaps.Esri.WorldImagery)
esri_sat.name = "ESRI Imagery"
m.add_layer(esri_sat)

# --- Overlay image ---
overlay = ImageOverlay(
    url=overlay_img_path,
    bounds=[(lat_min, lon_min), (lat_max, lon_max)],
    opacity=0.6,
    name="Oil Palm RGB"
)
m.add_layer(overlay)

# Add LayersControl after adding base and overlay layers
m.add_control(LayersControl(position="topright"))

# --- Debug output widget (outside the map) ---
debug_output = HTML(
    value="Debug info will appear here and in console",
    layout=Layout(width='100%', height='50px', border='1px solid gray', padding='5px')
)

# Create a layer for holding the marker
marker_layer = MarkerCluster(name="Selected Feature")
m.add_layer(marker_layer)

# --- Crown click handler ---
def handle_click(event=None, feature=None, id=None, properties=None):
    """Handle GeoJSON click events properly"""
    try:
        # Clear previous markers
        marker_layer.markers = []
        
        # Extract feature properties safely
        fid = None
        if feature is not None:
            fid = feature.get('id', None)
        if fid is None and properties and 'id' in properties:
            fid = properties['id']
        
        # Get geometry and calculate centroid
        geom = None
        lat, lon = None, None
        
        if feature is not None:
            geom = feature.get('geometry', None)
        
        if geom:
            try:
                geom_shape = shape(geom)
                centroid = geom_shape.centroid
                lat, lon = centroid.y, centroid.x
                
                # Print debug info
                debug_msg = f"Clicked ID: {fid}, Centroid: ({lat:.6f}, {lon:.6f})"
                print(debug_msg)
                debug_output.value = debug_msg
                
                # Create a popup message
                message = f"""
                <div style='background-color: white; padding: 8px; border-radius: 4px;'>
                    <h4 style='margin: 0 0 5px 0;'>Crown ID: {fid}</h4>
                    <p style='margin: 0;'>Centroid: ({lat:.6f}, {lon:.6f})</p>
                </div>
                """
                
                # Create a marker at the centroid
                marker = Marker(
                    location=(lat, lon),
                    draggable=False,
                    title=f"Crown ID: {fid}"
                )
                
                # Add popup to marker
                popup = Popup(
                    location=(lat, lon),
                    child=HTML(value=message),
                    close_button=True,
                    auto_pan=True,
                    min_width=200
                )
                marker.popup = popup
                
                marker_layer.markers = marker_layer.markers + (marker,)
                
            except Exception as e:
                print(f"Error processing geometry: {e}")
                debug_output.value = f"Error processing geometry: {e}"
    except Exception as e:
        print(f"Error in click handler: {e}")
        debug_output.value = f"Error in click handler: {e}"

# Create GeoJSON with appropriate settings to prevent layer issues
crown_layer = GeoJSON(
    data=crowns_json,
    name="Tree Crowns",
    hover_style={"fillColor": "red", "fillOpacity": 0.7},
    style={"fillOpacity": 0.5, "color": "yellow", "weight": 0.8},
    click_events=True  # Enable click events
)

# Register event handler properly
crown_layer.on_click(handle_click)
m.add_layer(crown_layer)


# --- Add instructions widget ---
instructions = HTML(
    value="""
    <div style="background-color: white; padding: 10px; border-radius: 5px; max-width: 300px;">
        <h3 style="margin-top: 0;">Oil Palm Crown Viewer</h3>
        <p><b>Instructions:</b></p>
        <ul>
            <li>Click on any crown polygon to see details</li>
        </ul>
    </div>
    """,
    layout=Layout(max_width='250px')
)

instructions_control = WidgetControl(
    widget=instructions,
    position='bottomleft'
)
m.add_control(instructions_control)

# Display the map and debug area
display(m)
display(debug_output)


### 2. Edit Geom

- https://geoman.io/docs/leaflet/modes/draw-mode
- https://geoman.io/docs/leaflet/modes/edit-mode

Hold Alt to disable snapping

In [ ]:
# --- Paths ---
site_path = "sample_data/"
img_path = site_path + "random_oilpalm.tif"
crowns_path = site_path + "filtered/filtered_by_knn_crowns.gpkg"
exported_crowns_path = site_path + "crowns_exported.gpkg"
overlay_img_path = site_path + "overlay_rgb.png"

In [ ]:
# --- Imports ---
import rasterio
import geopandas as gpd
import numpy as np
from shapely.geometry import box, mapping, shape, Polygon
from shapely.ops import transform
import pyproj
from PIL import Image
import json
from ipyleaflet import (
    Map, GeoJSON, GeomanDrawControl, Marker, Popup, Icon,
    basemap_to_tiles, basemaps, ImageOverlay, LayersControl, TileLayer,
    WidgetControl
)
from ipywidgets import HTML, Layout, VBox, Button

# --- Save RGB overlay as PNG from GeoTIFF ---
def save_rgb_overlay_as_png(img_path, output_img="overlay.png"):
    with rasterio.open(img_path) as src:
        img = src.read()
        bounds = src.bounds
        crs = src.crs
        
        if img.shape[0] >= 3:
            rgb = np.stack([img[0], img[1], img[2]], axis=-1)
        else:
            raise ValueError("Need at least 3 bands for RGB")
        
        # Normalize for better display
        rgb = rgb.astype(np.float32)
        rgb_min, rgb_max = np.percentile(rgb[rgb > 0], (2, 98))
        rgb = np.clip((rgb - rgb_min) / (rgb_max - rgb_min), 0, 1)
        rgb = (rgb * 255).astype(np.uint8)
        
        image = Image.fromarray(rgb)
        image.save(output_img)
        
        return output_img, bounds, crs

overlay_img_path, utm_bounds, utm_crs = save_rgb_overlay_as_png(img_path)

# --- Convert UTM bounds to WGS84 ---
def reproject_bounds_to_wgs84(bounds, src_crs):
    project = pyproj.Transformer.from_crs(src_crs, "EPSG:4326", always_xy=True).transform
    return transform(project, box(*bounds)).bounds

wgs84_bounds = reproject_bounds_to_wgs84(utm_bounds, utm_crs)
lon_min, lat_min, lon_max, lat_max = wgs84_bounds

# --- Load crowns and reproject to WGS84 ---
crowns = gpd.read_file(crowns_path)
if crowns.crs != "EPSG:4326":
    crowns = crowns.to_crs("EPSG:4326")

crowns["id"] = crowns.index
crowns_json = crowns.__geo_interface__

for i, feat in enumerate(crowns_json["features"]):
    feat["id"] = i

# --- Initialize Map ---
center_lat = (lat_min + lat_max) / 2
center_lon = (lon_min + lon_max) / 2

m = Map(center=(center_lat, center_lon), zoom=18, max_zoom=20)
m.layout.width = '100%'
m.layout.height = '900px'

# --- Base layers ---
google_sat = TileLayer(
    url="http://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}",
    attribution="Google Satellite",
    name="Google Satellite",
    max_zoom=20
)
m.add_layer(google_sat)

esri_sat = basemap_to_tiles(basemaps.Esri.WorldImagery)
esri_sat.name = "ESRI Imagery"
m.add_layer(esri_sat)

# --- Overlay image ---
overlay = ImageOverlay(
    url=overlay_img_path,
    bounds=[(lat_min, lon_min), (lat_max, lon_max)],
    opacity=0.6,
    name="Oil Palm RGB"
)
m.add_layer(overlay)

# Add LayersControl after adding base and overlay layers
m.add_control(LayersControl(position="topright"))

# --- Debug output widget (outside the map) ---
debug_output = HTML(
    value="Debug info will appear here and in console",
    layout=Layout(width='100%', height='50px', border='1px solid gray', padding='5px')
)

# Create GeoJSON with appropriate settings to prevent layer issues
crown_layer = GeoJSON(
    data=crowns_json,
    name="Tree Crowns",
    hover_style={"fillColor": "red", "fillOpacity": 0.7},
    style={"fillOpacity": 0.5, "color": "#6b92e5", "weight": 0.8},
    click_events=True  # Enable click events
)

m.add_layer(crown_layer)

draw_control = GeomanDrawControl()
draw_control.layers = [crown_layer]
draw_control.polyline =  {
    "pathOptions": {
        "color": "#a2e56b",
        "weight": 2,
        "opacity": 0.3
    }
}
draw_control.polygon = {
    "pathOptions": {
        "fillColor": "#dde56b",
        "color": "#6be5c3",
        "fillOpacity": 0.8
    }
}

def get_next_id(existing_ids):
    """Return the smallest non‑negative integer not in existing_ids."""
    existing = set(existing_ids)
    i = 0
    while True:
        if i not in existing:
            return i
        i += 1

def handle_draw(control, action, geo_json, **kwargs):
    """
    Single callback for all Geoman actions:
      - action: "create", "edit", "remove", etc.
      - geo_json: dict, list of dicts, or FeatureCollection
    """
    global crowns_json

    # 1) Normalize any payload into a list of feature dicts
    def as_feature_list(obj):
        if isinstance(obj, list):
            return obj
        if isinstance(obj, dict) and "features" in obj:
            return obj["features"]
        return [obj]

    if action == "create":
        new_feats = as_feature_list(geo_json)
        # find next available ID
        existing_ids = [f["id"] for f in crowns_json["features"]]
        for feat in new_feats:
            feat["id"] = get_next_id(existing_ids)
            existing_ids.append(feat["id"])
            crowns_json["features"].append(feat)
        new_id = feat["id"]
        total = len(crowns_json["features"])
        debug_output.value = f"Created a feature at ID :{new_id}, total feature: {total}"

    elif action == "remove":
        removed = as_feature_list(geo_json)
        removed_ids = {f["id"] for f in removed}
        before = len(crowns_json["features"])
        crowns_json["features"] = [
            feat for feat in crowns_json["features"]
            if feat["id"] not in removed_ids
        ]
        total = len(crowns_json["features"])
        debug_output.value = f"Removed a feature with ID: {sorted(removed_ids)}, total feature: {total}"

    else:
        debug_output.value = f"Unhandled action: {action}"

    # 4) Refresh on‑map layer
    crown_layer.data = crowns_json

draw_control.on_draw(handle_draw)

export_button = Button(
    description='💾 Export Crowns',
    button_style='success',
    layout=Layout(width='160px', height='40px')
)
export_button.on_click(lambda b: export_to_gpkg())

def export_to_gpkg():
    global crowns_json
    try:
        # Convert the edited GeoJSON to a GeoDataFrame
        crowns_gdf = gpd.GeoDataFrame.from_features(crowns_json)
        crowns_gdf.set_crs(epsg=4326, inplace=True)
        # Export the GeoDataFrame to a new GeoPackage file
        crowns_gdf.to_file(exported_crowns_path, driver='GPKG')
        debug_output.value = f'Exported new layer to: {exported_crowns_path}'
    except Exception as e:
        # Display an error message
        debug_output.value = f'Error: {e}'

# Add the button and output widget to the map
m.add_control(WidgetControl(widget=export_button, position='bottomright'))  

m.add_control(draw_control)
# Display the map and debug area
display(m)
display(debug_output)